
# Mapping Liverpool tracking data into gfootball compatible format : 

This notebook (a bite messy, sorry about that), maps the Liverpool tracking data to the Google Research Football format. Essentially, it changes the coordinates and the format. 

You can use it by plugging your tracking data at the fourth entry.


## Getting Liverpool data for one game : 

In [1]:
from matplotlib import pyplot as plt
import pandas as pd

import sys, os
sys.path.insert(0, os.path.abspath('../scripts/'))
import footyviz
import numpy as np

In [2]:
# For each play, we add to ball coordinates at each frame and for each player :
def _add_ball_coordinates(dataframe: pd.DataFrame) -> pd.DataFrame:
    
    list_of_plays = list(data.index.get_level_values('play').unique())
    
    #First, a function to compute this for one play at a time : 
    def _add_coord(df):
        # Getting the balls infos: 
        df_ball = df[df['edgecolor'] == 0]
        
        df_ball = df_ball[['frame','x','y','z']]
        df_ball.rename(columns = {'x': 'ball_x',
                                  'y': 'ball_y',
                                  'z':'ball_z'},inplace = True)
        
        df = df.merge(df_ball, on = 'frame', how = 'left')
        df = df.drop(columns = ['Unnamed: 0'])
        return df 
    
    for i,play in enumerate(list_of_plays):
        
        df = data.loc[play]
        df = df.reset_index()
        
        if i == 0:
            new_dataframe = _add_coord(df)
            new_dataframe['play'] = play
        else:
            df = _add_coord(df)
            df['play'] = play
            new_dataframe = pd.concat([new_dataframe,df])
            
    return new_dataframe
            

In [3]:
# We now add a boolean to mark the possession of the ball by a player
def _add_possession(dataframe: pd.DataFrame) -> pd.DataFrame:
    
    dataframe['possession'] = False
    columns = list(dataframe.columns.values)
    tab = dataframe.values 
    
    for line in tab:
        #Not ocmputing for the ball :
        if line[4] != 0:
            if ((line[8] - line[11])**2 + (line[9] - line[12])**2 < 1):
                line[-1] = True
                #Test to display later :
                line[4] = 'black'
                
    dataframe = pd.DataFrame(tab, columns = columns)
    return dataframe

In [4]:
data = pd.read_csv('../datasets/positional_data/liverpool_2019.csv', index_col=('play', 'frame'))
data.tail()

Unnamed: 0 bgcolor   dx   dy edgecolor  \
play                        frame                                           
Leicester 0 - [3] Liverpool 120          2745    blue  0.0  0.0     white   
                            121          2746    blue  0.0  0.0     white   
                            122          2747    blue  0.0  0.0     white   
                            123          2748    blue  0.0  0.0     white   
                            124          2749    blue  0.0  0.0     white   

                                   player  player_num     team          x  \
play                        frame                                           
Leicester 0 - [3] Liverpool 120     10267         NaN  defense  98.724826   
                            121     10267         NaN  defense  98.724826   
                            122     10267         NaN  defense  98.724826   
                            123     10267         NaN  defense  98.724826   
                            124     10267         NaN  defense  98.724826   

                                           y    z  
play                        frame                  
Leicester 0 - [3] Liverpool 120    53.720353  0.0  
                            121    53.720353  0.0  
                            122    53.720353  0.0  
                            123    53.720353  0.0  
                            124    53.720353  0.0

In [5]:
data['edgecolor'] = data['edgecolor'].fillna(0)

In [6]:
data_test = _add_ball_coordinates(data)
data_test = _add_possession(data_test)

In [16]:
play =  'Leicester 0 - [3] Liverpool'
df = data_test[data_test['play'] == play]
df = df.set_index('frame')
df.tail()

,bgcolor,dx,dy,edgecolor,player,player_num,team,x,y,z,ball_x,ball_y,ball_z,play,possession
frame,,,,,,,,,,,,,,,
120,blue,0,0,white,10267,NaN,defense,98.7248,53.7204,0,100.68,44.958,0,Leicester 0 - [3] Liverpool,False
121,blue,0,0,white,10267,NaN,defense,98.7248,53.7204,0,100.68,44.958,0,Leicester 0 - [3] Liverpool,False
122,blue,0,0,white,10267,NaN,defense,98.7248,53.7204,0,100.68,44.958,0,Leicester 0 - [3] Liverpool,False
123,blue,0,0,white,10267,NaN,defense,98.7248,53.7204,0,100.68,44.958,0,Leicester 0 - [3] Liverpool,False
124,blue,0,0,white,10267,NaN,defense,98.7248,53.7204,0,100.68,44.958,0,Leicester 0 - [3] Liverpool,False


# gfootball data : 

In [17]:
# The gfootball data takes the following form for each frame : 

In [9]:
replay_0 = replay_0 = {'debug': {'frame_cnt': 1, 'reward': 0, 'time': 11.043803118, 'config': {'action_set': 'full', 'custom_display_stats': None, 'display_game_stats': True, 'dump_full_episodes': True, 'dump_scores': False, 'players': ['keyboard:left_players=1'], 'level': '11_vs_11_stochastic', 'physics_steps_per_frame': 10, 'real_time': True, 'tracesdir': '/tmp/dumps', 'video_format': 'avi', 'video_quality_level': 0, 'write_video': False, 'episode_number': 2, 'reverse_team_processing': False, 'game_engine_random_seed': 1195427982}}, 'observation': {'ball': np.array([ 0.        , -0.        ,  0.10930105]), 'ball_direction': np.array([ 0.        , -0.        , -0.01213971]), 'ball_rotation': np.array([ 0., -0.,  0.]), 'left_team': np.array([[-1.00729287e+00,  2.44407050e-10],
       [ 3.90068977e-10,  1.55559480e-02],
       [ 5.17962500e-03, -2.32002642e-02],
       [-4.25872087e-01, -1.98561519e-01],
       [-5.02816439e-01, -6.25433624e-02],
       [-5.05514681e-01,  6.45929798e-02],
       [-4.24654722e-01,  1.98029920e-01],
       [-1.83394879e-01, -1.07413955e-01],
       [-2.65282214e-01,  1.79797735e-05],
       [-1.85588703e-01,  1.07415758e-01],
       [-9.70329251e-03, -2.19043002e-01]]), 'left_team_direction': np.array([[ 4.46333596e-03,  2.32619021e-10],
       [ 1.56752056e-10, -3.67024867e-03],
       [ 4.07242170e-03, -2.04267120e-03],
       [ 2.56580464e-03,  1.28244096e-03],
       [ 3.17236967e-03,  2.11017556e-03],
       [ 0.00000000e+00, -0.00000000e+00],
       [ 3.21584591e-03, -1.45674893e-03],
       [ 4.07643197e-03,  5.58971806e-05],
       [ 5.09514520e-03,  2.93488465e-05],
       [ 2.60559958e-03,  5.40719666e-05],
       [ 2.45425384e-04,  1.45657326e-03]]), 'left_team_tired_factor': np.array([1.32441521e-04, 2.83360481e-04, 2.62439251e-04, 3.67164612e-05,
       1.66952610e-04, 0.00000000e+00, 9.24468040e-05, 1.07288361e-04,
       1.92224979e-04, 2.31266022e-05, 1.53183937e-05]), 'left_team_active': np.array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'left_team_yellow_card': np.array([False, False, False, False, False, False, False, False, False,
       False, False]), 'left_team_roles': np.array([0, 7, 9, 2, 1, 1, 3, 5, 5, 5, 6]), 'right_team': np.array([[ 1.00640988e+00, -3.02056574e-11],
       [ 4.43515852e-02,  1.23523269e-03],
       [ 1.33955395e-02, -2.16839880e-01],
       [ 4.26654398e-01,  1.98944613e-01],
       [ 5.02162635e-01,  6.17753938e-02],
       [ 5.05514681e-01, -6.45929798e-02],
       [ 4.23637688e-01, -1.97564706e-01],
       [ 1.81663871e-01,  1.03803135e-01],
       [ 2.70525187e-01,  0.00000000e+00],
       [ 1.81629047e-01, -1.05274908e-01],
       [ 9.67132300e-03,  2.16276765e-01]]), 'right_team_direction': np.array([[-4.68702894e-03, -2.25887625e-11],
       [-2.71152915e-03,  2.57208757e-03],
       [ 2.62233778e-03,  2.51533091e-03],
       [-0.00000000e+00,  0.00000000e+00],
       [-3.02651338e-03, -2.35213037e-03],
       [-0.00000000e+00,  0.00000000e+00],
       [-3.56470840e-03,  1.61280471e-03],
       [-3.01774801e-03, -2.96688545e-03],
       [-0.00000000e+00,  0.00000000e+00],
       [-4.35972912e-03,  1.71638560e-03],
       [-5.92967612e-04, -3.18545476e-03]]), 'right_team_tired_factor': np.array([0.00016373, 0.00026643, 0.0002293 , 0.        , 0.00022024,
       0.        , 0.00013959, 0.00027096, 0.        , 0.00019979,
       0.00017744]), 'right_team_active': np.array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'right_team_yellow_card': np.array([False, False, False, False, False, False, False, False, False,
       False, False]), 'right_team_roles': np.array([0, 7, 9, 2, 1, 1, 3, 5, 5, 5, 6]), 'left_agent_sticky_actions': [np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])], 'left_agent_controlled_player': [1], 'right_agent_sticky_actions': [], 'right_agent_controlled_player': [], 'game_mode': 0, 'score': [0, 0], 'ball_owned_team': -1, 'ball_owned_player': -1, 'steps_left': 2999}, 'reward': 0, 'cumulative_reward': 0}

We mainly need to change a few things : 

* Ball positions and velocity
* Players positions and velocity 
* Who owns the ball, and what team he is in 

We also need to change the scale: 

Our data are in the following square :

* (0,100) --------------------- (100,100)
*  |
*  |
*  |
*  |
*  |
* (0,0) ------------------------ (100,0)

and google's : 

* (-1,-1) --------------------- (1,-1)
*  |
*  |
*  |
*  |
*  |
* (-1,1) ------------------------ (1,1)

In [19]:
def _scale_mapper(x,y):
    #Takes our x,y and maps it into google's (x,y)
    new_x = (x/100. - 0.5)*2
    new_y = -(y/100. - 0.5)*2
    return (new_x,new_y)
     
    

In [21]:
df.tail()

,bgcolor,dx,dy,edgecolor,player,player_num,team,x,y,z,ball_x,ball_y,ball_z,play,possession
frame,,,,,,,,,,,,,,,
120,blue,0,0,white,10267,NaN,defense,98.7248,53.7204,0,100.68,44.958,0,Leicester 0 - [3] Liverpool,False
121,blue,0,0,white,10267,NaN,defense,98.7248,53.7204,0,100.68,44.958,0,Leicester 0 - [3] Liverpool,False
122,blue,0,0,white,10267,NaN,defense,98.7248,53.7204,0,100.68,44.958,0,Leicester 0 - [3] Liverpool,False
123,blue,0,0,white,10267,NaN,defense,98.7248,53.7204,0,100.68,44.958,0,Leicester 0 - [3] Liverpool,False
124,blue,0,0,white,10267,NaN,defense,98.7248,53.7204,0,100.68,44.958,0,Leicester 0 - [3] Liverpool,False


In [23]:
df = df.reset_index()

In [24]:
df[df['frame'] == 0]

,frame,bgcolor,dx,dy,edgecolor,player,player_num,team,x,y,z,ball_x,ball_y,ball_z,play,possession
0,0,NaN,0,0,0,0,NaN,NaN,80.2552,39.0475,0,80.2552,39.0475,0,Leicester 0 - [3] Liverpool,False
125,0,red,0,0,white,13,9,attack,74.8638,38.6718,0,80.2552,39.0475,0,Leicester 0 - [3] Liverpool,False
250,0,red,0,0,white,65,66,attack,77.7743,8.86703,0,80.2552,39.0475,0,Leicester 0 - [3] Liverpool,False
375,0,blue,0,0,white,379,NaN,defense,83.6547,56.6344,0,80.2552,39.0475,0,Leicester 0 - [3] Liverpool,False
500,0,blue,0,0,white,380,NaN,defense,86.0106,49.6209,0,80.2552,39.0475,0,Leicester 0 - [3] Liverpool,False
625,0,blue,0,0,white,381,NaN,defense,84.4115,41.1635,0,80.2552,39.0475,0,Leicester 0 - [3] Liverpool,False
750,0,blue,0,0,white,382,NaN,defense,84.0833,28.3894,0,80.2552,39.0475,0,Leicester 0 - [3] Liverpool,False
875,0,blue,0,0,white,1330,NaN,defense,76.7885,28.766,0,80.2552,39.0475,0,Leicester 0 - [3] Liverpool,False
1000,0,red,0,0,white,1848,10,attack,77.4185,52.4303,0,80.2552,39.0475,0,Leicester 0 - [3] Liverpool,False
1125,0,red,0,0,white,2067,27,attack,81.6812,75.094,0,80.2552,39.0475,0,Leicester 0 - [3] Liverpool,False


In [25]:
full_info = []

for i in list(df['frame'].unique()):
    temp_df = df[df['frame'] == i]
    if (i+1) in list(df['frame'].unique()):
        next_temp_df = df[df['frame'] == i+1]
    else:
        next_temp_df = df[df['frame'] == i]
    
    left_df = temp_df[temp_df['team'] == 'attack']
    right_df = temp_df[temp_df['team'] == 'defense']
    
    next_left_df = next_temp_df[next_temp_df['team'] == 'attack']
    next_right_df = next_temp_df[next_temp_df['team'] == 'defense']
    
    left_team_positions = [[-1.,0.]]
    right_team_positions = []
    left_team_velocity = [[0.,0.]]
    right_team_velocity = []
    ball_position = []
    ball_velocity = []
    ball_team_owner = -1
    ball_player_owner = -1
    
    # Ball Information : 
    
    
    
    #Left team informations : 
    
    for indx,line in enumerate(left_df.values):
        new_x,new_y = _scale_mapper(line[8],line[9])
        next_line = next_left_df.values[indx]
        next_new_x,next_new_y = _scale_mapper(next_line[8],next_line[9])
        dx,dy = next_new_x - new_x, next_new_y-new_y
        
        left_team_positions.append([new_x,new_y])
        left_team_velocity.append([dx,dy])
        
        if line[-1] == True and ball_team_owner == -1:
            ball_team_owner = 0
            ball_player_owner = (indx+1)
            
        if indx == 0:
            #Ball information : 
            ball_x, ball_y = _scale_mapper(line[11],line[12])
            next_ball_x, next_ball_y = _scale_mapper(next_line[11],next_line[12])
            dx,dy = next_ball_x - ball_x, next_ball_y-ball_y
            ball_z = line[13]
            
            ball_position = [ball_x,ball_y,ball_z]
            ball_velocity = [dx,dy,0.0]
            
    for indx,line in enumerate(right_df.values):
        new_x,new_y = _scale_mapper(line[8],line[9])
        next_line = next_right_df.values[indx]
        next_new_x,next_new_y = _scale_mapper(next_line[8],next_line[9])
        dx,dy = next_new_x - new_x, next_new_y-new_y
        
        right_team_positions.append([new_x,new_y])
        right_team_velocity.append([dx,dy])

    info = {
        'left_team_positions':left_team_positions,
    'right_team_positions':right_team_positions,
    'left_team_velocity':left_team_velocity,
    'right_team_velocity': right_team_velocity,
    'ball_position': ball_position,
    'ball_velocity': ball_velocity,
    'ball_team_owner': ball_team_owner,
    'ball_player_owner': ball_player_owner
    }
    
    print(info)
    full_info.append(info)

{'left_team_positions': [[-1.0, 0.0], [0.49727548777794484, 0.2265630140677808], [0.5554869947125332, 0.8226593127200806], [0.5483705748675209, -0.048606001929745846], [0.6336248769890389, -0.5018808007766073], [0.6051047143117816, 0.2190494358713021], [0.6598945902516942, 0.1375078060239363], [0.11564625850340127, 0.26050420168067223], [0.22721088435374148, -0.3739495798319328], [0.0748299319727892, -0.06722689075630295], [0.2897959183673471, -0.6596638655462186]], 'right_team_positions': [[0.6730946749659676, -0.13268717456899903], [0.7202118880204429, 0.0075818513731165105], [0.6882302244668161, 0.17672966407663193], [0.6816658930982351, 0.432212348370131], [0.535769583161307, 0.42467918285337436], [0.5389993995559277, 0.24409285437068218], [0.49672105913855, 0.1431307891866005], [0.4224263489232407, -0.550239889831672], [0.2979591836734692, -0.3739495798319328], [0.18095238095238075, 0.04201680672268926], [0.9522339419227024, 0.026476594331903747]], 'left_team_velocity': [[0.0, 0.0

{'left_team_positions': [[-1.0, 0.0], [0.5893716644605709, 0.13530962969798066], [0.6217352269369025, 0.7768776896005812], [0.6243011375752916, -0.06361661096924198], [0.6722824357508992, -0.4900689655281456], [0.6657144420535785, 0.2765350531004134], [0.7369115761684986, 0.10634923172502397], [0.11564625850340149, 0.26050420168067223], [0.24571428571428577, -0.35381970970206256], [0.0748299319727892, -0.06722689075630295], [0.3196536796536802, -0.6440794499618028]], 'right_team_positions': [[0.6969331881137553, -0.10252523473635788], [0.7806565878169338, 0.06790164803749121], [0.7185534910565199, 0.2231113273503551], [0.6973958510262463, 0.3085710004001345], [0.5946610921582829, 0.41985669133689385], [0.6107672528385373, 0.23781691950771933], [0.5727191884483063, 0.124828264901508], [0.47173573389107526, -0.5286590425295339], [0.32529375386518233, -0.3486249045072578], [0.18095238095238075, 0.04201680672268926], [0.9592436814818388, 0.023004796508612202]], 'left_team_velocity': [[0.0,

{'left_team_positions': [[-1.0, 0.0], [0.7509886619865487, 0.02502536850981174], [0.8161307554940729, 0.6781040003533756], [0.7818123545280102, -0.05363600616558295], [0.7769032827706914, -0.36824148641728205], [0.7034343079730059, 0.26960702735599984], [0.8441657488987984, 0.13036428911759512], [0.11564625850340193, 0.26050420168067223], [0.28054421768707494, -0.31592818945760137], [0.0748299319727892, -0.06722689075630295], [0.37585652442795325, -0.6147440794499619]], 'right_team_positions': [[0.8179170227121313, -0.052451672912090075], [0.874362561305492, 0.10288257097703801], [0.8414801223856969, 0.23433363706214194], [0.772087482486151, 0.3002779555022471], [0.7781955076490712, 0.448782842231643], [0.6951101499437575, 0.28750146269530585], [0.7121032783545889, 0.09751130353178372], [0.6007935888302485, -0.46956491069254036], [0.37674706246134804, -0.3009549274255159], [0.18095238095238075, 0.04201680672268904], [0.9717962922955139, 0.05271897329472863]], 'left_team_velocity': [[0.

{'left_team_positions': [[-1.0, 0.0], [0.8548075409119165, -0.07163988896061158], [0.9311608858410989, 0.6233049367994961], [0.8854917949618855, 0.04823924940660007], [0.8531762484949497, -0.2903851333740832], [0.7296671046508072, 0.25135078100375696], [0.9145120310007335, 0.1791249468428069], [0.11564625850340127, 0.2605042016806727], [0.3142857142857145, -0.27922077922077926], [0.07482993197278898, -0.06722689075630228], [0.43030303030302997, -0.5863254392666155]], 'right_team_positions': [[0.9037774539059311, 0.0019990988175186963], [0.9131978166311159, 0.11844983871421122], [0.9259067176377114, 0.22658572373530295], [0.8021403306819774, 0.3403550967299739], [0.9012032512454773, 0.45027521332225984], [0.7780416223328341, 0.260900142587435], [0.8488680492577962, 0.0670658007498135], [0.7281786240218804, -0.4118310150141389], [0.4265924551638838, -0.25477463712757786], [0.18095238095238075, 0.04201680672268926], [0.9805844931756817, 0.010773744250684114]], 'left_team_velocity': [[0.0,

{'left_team_positions': [[-1.0, 0.0], [0.8595685264739004, -0.13463961574155414], [0.959183673469387, 0.5], [0.9062538404047973, 0.012372386475637542], [0.884684658055233, -0.24485563437320912], [0.7066495877821817, 0.2108061178550501], [0.8937822060267344, 0.2047416865010735], [0.11564625850340127, 0.26050420168067223], [0.34693877551020424, -0.24369747899159666], [0.0748299319727892, -0.06722689075630295], [0.4829931972789112, -0.5588235294117647]], 'right_team_positions': [[0.9058010880456784, -0.09406070180596715], [0.9102520210834728, 0.03767319166666794], [0.9206366933604875, 0.103388223419695], [0.8069755992959591, 0.3301643579523238], [0.8880107265219157, 0.444572001368432], [0.8020950986877184, 0.2109434651317088], [0.8767372452560898, -0.04309675699223936], [0.8268349711898639, -0.3371610732463217], [0.4748299319727889, -0.21008403361344552], [0.18095238095238075, 0.04201680672268926], [0.9744965230886646, -0.07440706940773634]], 'left_team_velocity': [[0.0, 0.0], [0.0, 0.0],

In [26]:
import pickle
import six
with open('full_info_liverpool_leicester.dump', "wb") as fh:
        pickle.dump((full_info,), fh, pickle.HIGHEST_PROTOCOL)

In [27]:
def load_dump(dump_file):
    dump = []
    with open(dump_file, 'rb') as in_fd:
      while True:
        try:
          step = six.moves.cPickle.load(in_fd)
        except EOFError:
          return dump
        dump.append(step)
    return dump

In [28]:
test = load_dump('full_info_liverpool_leicester.dump')

In [29]:
test[0][0][1]

{'left_team_positions': [[-1.0, 0.0],
  [0.49727548777794484, 0.2265630140677808],
  [0.5554869947125332, 0.8226593127200806],
  [0.5483705748675209, -0.048606001929745846],
  [0.6336248769890389, -0.5018808007766073],
  [0.6051047143117816, 0.2190494358713021],
  [0.6598945902516942, 0.1375078060239363],
  [0.11564625850340127, 0.26050420168067223],
  [0.22721088435374148, -0.3739495798319328],
  [0.0748299319727892, -0.06722689075630295],
  [0.2897959183673471, -0.6596638655462186]],
 'right_team_positions': [[0.6730946749659676, -0.13268717456899903],
  [0.7202118880204429, 0.0075818513731165105],
  [0.6882302244668161, 0.17672966407663193],
  [0.6816658930982351, 0.432212348370131],
  [0.535769583161307, 0.42467918285337436],
  [0.5389993995559277, 0.24409285437068218],
  [0.49672105913855, 0.1431307891866005],
  [0.4224263489232407, -0.550239889831672],
  [0.2979591836734692, -0.3739495798319328],
  [0.18095238095238075, 0.04201680672268926],
  [0.9522339419227024, 0.0264765943319